<img src="NotebookAddons/blackboard-banner.png" width="100%" />

# Tile Data Stack to MGRS (Military Grid Reference System)

<img style="padding: 7px" src="NotebookAddons/UAFLogo_A_647.png" width="170" align="right"/></font>

**Alex Lewandowski; University of Alaska Fairbanks**

This notebook subsets a tiff stack into MGRS tiles

### Important Note about JupyterHub

**Your JupyterHub server will automatically shutdown when left idle for more than 1 hour. Your notebooks will not be lost but you will have to restart their kernels and re-run them from the beginning. You will not be able to seamlessly continue running a partially run notebook.**


In [ ]:

%%javascript
var kernel = Jupyter.notebook.kernel;
var command = ["notebookUrl = ",
               "'", window.location, "'" ].join('')
kernel.execute(command)

In [ ]:
from IPython.display import Markdown
from IPython.display import display

user = !echo $JUPYTERHUB_USER
env = !echo $CONDA_PREFIX
if env[0] == '':
    env[0] = 'Python 3 (base)'
if env[0] != '/home/jovyan/.local/envs/rtc_analysis':
    display(Markdown(f'<text style=color:red><strong>WARNING:</strong></text>'))
    display(Markdown(f'<text style=color:red>This notebook should be run using the "rtc_analysis" conda environment.</text>'))
    display(Markdown(f'<text style=color:red>It is currently using the "{env[0].split("/")[-1]}" environment.</text>'))
    display(Markdown(f'<text style=color:red>Select the "rtc_analysis" from the "Change Kernel" submenu of the "Kernel" menu.</text>'))
    display(Markdown(f'<text style=color:red>If the "rtc_analysis" environment is not present, use <a href="{notebookUrl.split("/user")[0]}/user/{user[0]}/notebooks/conda_environments/Create_OSL_Conda_Environments.ipynb"> Create_OSL_Conda_Environments.ipynb </a> to create it.</text>'))
    display(Markdown(f'<text style=color:red>Note that you must restart your server after creating a new environment before it is usable by notebooks.</text>'))

## 0. Import Relevant Python Packages

We will use the following scientific library:
- [GDAL](https://www.gdal.org/) is a software library for reading and writing raster and vector geospatial data formats. It includes a collection of programs tailored for geospatial data processing. Most modern GIS systems (such as ArcGIS or QGIS) use GDAL in the background.

**mport the necesssary libraries and modules:**

In [ ]:
try:
    import mgrs
except ModuleNotFoundError:
    %conda install -c conda-forge mgrs 

In [ ]:
%%capture
from pathlib import Path
import glob
import json # for loads
import re
import shutil

import mgrs
from mgrs.core import MGRSError
from osgeo import gdal
import pyproj 

from IPython.display import Markdown
from IPython.display import display

%matplotlib notebook
import matplotlib.pyplot as plt 
plt.rcParams.update({'font.size': 12})

import asf_notebook as asfn
asfn.jupytertheme_matplotlib_format()

**Write functions to gather and print individual tiff paths:**

In [ ]:
def get_tiff_paths(paths):
    tiff_paths = list(Path('.').glob(paths))
    tiff_paths.sort()
    return tiff_paths

def print_tiff_paths(tiff_paths):
    print("Tiff paths:")
    for p in tiff_paths:
        print(f"{p}\n")

**Enter the path to the directory holding your tiffs:**

In [ ]:
while True:
    print("Enter the absolute path to the directory holding your tiffs.")
    tiff_dir = Path(input()).relative_to(Path.cwd())
    paths = f"{tiff_dir}/*.tif*"
    if tiff_dir.is_dir():
        tiff_paths = get_tiff_paths(paths)
        if len(tiff_paths) < 1:
            print(f"{tiff_dir} exists but contains no tifs.")
            print("You will not be able to proceed until tifs are prepared.")
        break
    else:
        print(f"\n{tiff_dir} does not exist.")
        continue

**Determine the path to the analysis directory containing the tiff directory:**

In [ ]:
analysis_dir = Path.cwd()/tiff_dir
print(analysis_dir)

## 1. Subset to MGRS Tiles

**Write a function to find the MGRS corner coordinates of a geotiff**

In [ ]:
def get_mgrs_corners(product_path):
    info = gdal.Info(str(product_path), format='json')
    coord_sys = info['coordinateSystem']['wkt']
    utm_regex = '(?<=UTM zone )[0-9]{1,2}[A-Z]'
    try:
        utm = re.search(utm_regex, coord_sys).group(0)
    except AttributeError:
        raise(f"Unable to parse UTM zone from {coord_sys}")
    
    corners = info['cornerCoordinates']
    ulx = corners['upperLeft'][0]
    uly = corners['upperLeft'][1]
    lrx = corners['lowerRight'][0]
    lry = corners['lowerRight'][1]
    
    corners = dict()
    m = mgrs.MGRS()
    corners.update({'ul': m.UTMToMGRS(int(utm[:-1]), utm[-1], ulx, uly, MGRSPrecision=2)})
    corners.update({'ur': m.UTMToMGRS(int(utm[:-1]), utm[-1], lrx, uly, MGRSPrecision=2)})
    corners.update({'ll': m.UTMToMGRS(int(utm[:-1]), utm[-1], ulx, lry, MGRSPrecision=2)})
    corners.update({'lr': m.UTMToMGRS(int(utm[:-1]), utm[-1], lrx, lry, MGRSPrecision=2)})
    return corners

**Subset the tiffs by MGRS tile, saving them into directories named `<analysis_directory>/<UTM tile><MGRS tile>/`**

In [ ]:
tile_side_len = 100000
for tif in tiff_paths:
    print(tif)
    mgrs_corners = get_mgrs_corners(tif)
    print(mgrs_corners)

    m = mgrs.MGRS() 
    utm_corners = {k: m.MGRSToUTM(v) for (k, v) in mgrs_corners.items()}
    
    x_tile_count = int((utm_corners['ur'][2] - utm_corners['ul'][2]) / tile_side_len) + 1
    y_tile_count = int((utm_corners['ul'][3] - utm_corners['ll'][3]) / tile_side_len) + 1
    print(x_tile_count)
    print(y_tile_count)
    
    
    utm_x = utm_corners['ll'][2]
    for x in range(0, x_tile_count):
        utm_y = utm_corners['ll'][3]
        for y in range(0, y_tile_count):
            mgrs_tile = m.UTMToMGRS(utm_corners['ll'][0], utm_corners['ll'][1], 
                                    utm_x, utm_y,
                                    MGRSPrecision=2)

            mgrs_tile_dir = analysis_dir/mgrs_tile
            if not mgrs_tile_dir.is_dir():
                mgrs_tile_dir.mkdir()
            ulx = utm_x - 1
            uly = utm_y + tile_side_len + 1
            lrx = utm_x + tile_side_len + 1
            lry = utm_y - 1
            print(f"MGRS tile: {mgrs_tile}")
            print(f"ul: [{ulx}, {uly}]")
            print(f"lr: [{lrx}, {lry}]")
            out = f"{mgrs_tile_dir}/{tif.stem}_{mgrs_tile}.tif"
            gdal.Translate(destName=out, srcDS=str(tif), projWin=[ulx, uly, lrx, lry])
            print(gdal.Info(out, format='json')['size'])
            utm_y += tile_side_len
        utm_x += tile_side_len
    print()

**Print the path to the directory holding your MGRS subset directories:**

In [ ]:
print(analysis_dir)

*MGRS_Tile_Data_Stack - Version 0.0.1 - September 2021*